# Criação da baseline

- Qual a métrica que iremos utilizar para avaliar qual será o melhor modelo?
    - Acurácia
    - Recall
    - Precision
    - F1-score

## 0. Setup

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder

## 1. Carregando os dados

In [2]:
dados = pd.read_csv(filepath_or_buffer = '../data/raw/aug_train.csv')

dados.head(3)

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,8949,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,29725,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,11561,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


## 2. Manuseio dos dados

### 2.1. Exclusão da variável enrollee_id

In [3]:
dados = dados.drop(columns = 'enrollee_id', axis = 1)

dados.head(3)

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,city_103,0.920,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,NaN,NaN,1,36,1.0
1,city_40,0.776,Male,No relevent experience,no_enrollment,Graduate,STEM,15,50-99,Pvt Ltd,>4,47,0.0
2,city_21,0.624,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,83,0.0


In [4]:
#---- Selecionar as variáveis do banco de dados que são qualitativas

le = LabelEncoder()

list_colunas_qual = list(dados.select_dtypes(include = ['object']).columns)

dados[list_colunas_qual] = dados[list_colunas_qual].apply(lambda x: le.fit_transform(x.values))

In [5]:
dados.head(3)

,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,5,0.920,1,0,2,0,5,21,8,6,0,36,1.0
1,77,0.776,1,1,2,0,5,6,4,5,4,47,0.0
2,64,0.624,3,1,0,0,5,15,8,6,5,83,0.0


## 3. Criando a baseline

### 3.1. Divisão da base de treino e teste

In [6]:
X = dados.drop(columns = 'target', axis = 1)

y = dados.target

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.7, random_state = 19, stratify = y)

In [8]:
print(f'Quantidade de linhas do X_train: {X_train.shape[0]} \n \
Quantidade de linhas do X_test: {X_test.shape[0]}\n \
Quantidade de linhas do y_train: {y_train.shape[0]}\n \
Quantidade de linhas do y_test: {y_test.shape[0]}\
')

Quantidade de linhas do X_train: 13410 
 Quantidade de linhas do X_test: 5748
 Quantidade de linhas do y_train: 13410
 Quantidade de linhas do y_test: 5748


### 3.2. Escolha do modelo

In [9]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state = 19)

In [10]:
lr.fit(X_train, y_train)

/home/rafael/Documentos/Github/hr_analysis/env/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=19)